<a href="https://colab.research.google.com/github/SantiagoVieiura/Lab1-EDD2/blob/main/laboratorio_EDD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

def leerArchivo(nombre):
  df = pd.read_csv(nombre)
  return df




In [ ]:
from graphviz import Digraph
import csv

class Nodo:
    def __init__(self, ID, Pais, ISO3, T_list, T_media):
        self.ID = ID
        self.Pais = Pais
        self.ISO3 = ISO3
        self.T_list = T_list
        self.T_media = T_media
        self.izquierda = None
        self.derecha = None
        self.padre = None

class AVL:
    def __init__(self):
        self.raiz = None

    def obtener_altura(self, nodo):
        if not nodo:
            return 0
        return 1 + max(self.obtener_altura(nodo.izquierda),
                       self.obtener_altura(nodo.derecha))

    def mostrar_nivel(self, nodo, nivel):
        if not nodo:
            return
        if nivel == 1:
            print(nodo.ISO3, end=" ")
        else:
            self.mostrar_nivel(nodo.izquierda, nivel - 1)
            self.mostrar_nivel(nodo.derecha, nivel - 1)

    def recorrido_por_niveles(self, nodo=None, nivel=1, h=None):
        if nodo is None:
            nodo = self.raiz
        if h is None:
            h = self.obtener_altura(self.raiz)

        if nivel > h:
            return

        self.mostrar_nivel(nodo, nivel)
        self.recorrido_por_niveles(self.raiz, nivel + 1, h)

    def obtener_balance(self, nodo):
        if not nodo:
            return 0
        return self.obtener_altura(nodo.izquierda) - self.obtener_altura(nodo.derecha)

    def rotacion_derecha(self, nodo):
        new = nodo.izquierda
        Temp = new.derecha
        new.derecha = nodo
        nodo.izquierda = Temp
        return new

    def rotacion_izquierda(self, nodo):
        new = nodo.derecha
        Temp = new.izquierda
        new.izquierda = nodo
        nodo.derecha = Temp
        return new

    def buscar(self, nodo, ISO, p, a):
        if not nodo:
            return None, None, None
        if ISO == nodo.ISO3:
            return nodo, p, a
        encontrado, p, a = self.buscar(nodo.izquierda, ISO, nodo, p)
        if encontrado:
            return encontrado, p, a
        return self.buscar(nodo.derecha, ISO, nodo, p)

    def buscar_ISO(self, ISO):
        return self.buscar(self.raiz, ISO, None, None)

    def insertar(self, nodo, ID, Pais, ISO3, T_list, T_media):
        if not nodo:
            return Nodo(ID, Pais, ISO3, T_list, T_media)

        if T_media < nodo.T_media:
            nodo.izquierda = self.insertar(nodo.izquierda, ID, Pais, ISO3, T_list, T_media)
        elif T_media > nodo.T_media:
            nodo.derecha = self.insertar(nodo.derecha, ID, Pais, ISO3, T_list, T_media)
        else:
            return nodo

        balance = self.obtener_balance(nodo)

        if balance > 1 and T_media < nodo.izquierda.T_media:
            return self.rotacion_derecha(nodo)

        if balance < -1 and T_media > nodo.derecha.T_media:
            return self.rotacion_izquierda(nodo)

        if balance > 1 and T_media > nodo.izquierda.T_media:
            nodo.izquierda = self.rotacion_izquierda(nodo.izquierda)
            return self.rotacion_derecha(nodo)

        if balance < -1 and T_media < nodo.derecha.T_media:
            nodo.derecha = self.rotacion_derecha(nodo.derecha)
            return self.rotacion_izquierda(nodo)

        return nodo

    def insertar_nodo(self, ID, Pais, ISO3, T_list, T_media):
        nodo, p, a = self.buscar_ISO(ISO3)
        if nodo:
            print("El acrónimo " + ISO3 + " ya existe.")
            return
        self.raiz = self.insertar(self.raiz, ID, Pais, ISO3, T_list, T_media)
        self.mostrar("Avl_Tree")
        print("Nodo con acrónimo " + ISO3 + " insertado.")

    def _min_valor(self, nodo):
        actual = nodo
        while actual.izquierda:
            actual = actual.izquierda
        return actual

    def _eliminar(self, nodo, T_media):
        if not nodo:
            return nodo

        if T_media < nodo.T_media:
            nodo.izquierda = self._eliminar(nodo.izquierda, T_media)
        elif T_media > nodo.T_media:
            nodo.derecha = self._eliminar(nodo.derecha, T_media)
        else:
            if not nodo.izquierda:
                return nodo.derecha
            elif not nodo.derecha:
                return nodo.izquierda

            sucesor = self._min_valor(nodo.derecha)
            nodo.ID = sucesor.ID
            nodo.Pais = sucesor.Pais
            nodo.ISO3 = sucesor.ISO3
            nodo.T_list = sucesor.T_list
            nodo.T_media = sucesor.T_media
            nodo.derecha = self._eliminar(nodo.derecha, sucesor.T_media)

        balance = self.obtener_balance(nodo)

        if balance > 1 and self.obtener_balance(nodo.izquierda) >= 0:
            return self.rotacion_derecha(nodo)
        if balance > 1 and self.obtener_balance(nodo.izquierda) < 0:
            nodo.izquierda = self.rotacion_izquierda(nodo.izquierda)
            return self.rotacion_derecha(nodo)
        if balance < -1 and self.obtener_balance(nodo.derecha) <= 0:
            return self.rotacion_izquierda(nodo)
        if balance < -1 and self.obtener_balance(nodo.derecha) > 0:
            nodo.derecha = self.rotacion_derecha(nodo.derecha)
            return self.rotacion_izquierda(nodo)

        return nodo

    def eliminar_ISO(self, ISO):
        nodo, p, a = self.buscar_ISO(ISO)
        if not nodo:
            print("No se encontró el acrónimo " + ISO + ".")
            return
        self.raiz = self._eliminar(self.raiz, nodo.T_media)
        avl.mostrar("Avl_Tree")
        print("Nodo con acrónimo " + ISO + " eliminado.")

    def graficar(self, nodo, dot):
        if not nodo:
            return
        etiqueta = f"{nodo.ISO3}\n({nodo.T_media})"
        dot.node(str(id(nodo)), etiqueta, shape="circle", style="filled", color="lightblue")

        if nodo.izquierda:
            dot.edge(str(id(nodo)), str(id(nodo.izquierda)))
            self.graficar(nodo.izquierda, dot)
        if nodo.derecha:
            dot.edge(str(id(nodo)), str(id(nodo.derecha)))
            self.graficar(nodo.derecha, dot)

    def mostrar(self, nombre="avl"):
        dot = Digraph(name="AVL", format="png")
        if self.raiz:
            self.graficar(self.raiz, dot)
        dot.render(nombre, view=True)

    def recorrer_inorden(self, nodo, lista):
        if not nodo:
            return
        self.recorrer_inorden(nodo.izquierda, lista)
        lista.append(nodo)
        self.recorrer_inorden(nodo.derecha, lista)

    def obtener_todos_nodos(self):
        lista = []
        self.recorrer_inorden(self.raiz, lista)
        return lista

    def buscar_mayores_promedio_anual(self, year_idx):
        nodos = self.obtener_todos_nodos()
        suma = 0
        cantidad = 0
        for n in nodos:
            if year_idx < len(n.T_list):
                suma += n.T_list[year_idx]
                cantidad += 1
        if cantidad == 0:
            return []
        promedio = suma / cantidad
        resultado = []
        for n in nodos:
            if year_idx < len(n.T_list) and n.T_list[year_idx] > promedio:
                resultado.append(n)
        return resultado

    def buscar_menores_promedio_global(self, year_idx):
        nodos = self.obtener_todos_nodos()
        suma = 0
        cantidad = 0
        for n in nodos:
            for valor in n.T_list:
                suma += valor
                cantidad += 1
        if cantidad == 0:
            return []
        promedio_global = suma / cantidad
        resultado = []
        for n in nodos:
            if year_idx < len(n.T_list) and n.T_list[year_idx] < promedio_global:
                resultado.append(n)
        return resultado

    def buscar_media_mayor_que(self, limite):
        nodos = self.obtener_todos_nodos()
        resultado = []
        for n in nodos:
            if n.T_media >= limite:
                resultado.append(n)
        return resultado

    def tio(self, nodo, abuelo):
        if not abuelo:
            return None
        if abuelo.izquierda == nodo:
            return abuelo.derecha
        else:
            return abuelo.izquierda



def mostrar_nodo(nodo):
    if nodo:
        print(f"ID: {nodo.ID}, País: {nodo.Pais}, ISO3: {nodo.ISO3}, "
              f"T_media: {nodo.T_media:.2f}")
    else:
        print("Nodo no encontrado.")

def menu():
    avl = AVL()
    dataset = []

    with open("/content/dataset_climate_change.csv", newline="") as f:
        reader = csv.reader(f)
        header = next(reader)

        start_year = "F1961"
        end_year   = "F2022"
        start_idx = header.index(start_year)
        end_idx   = header.index(end_year)

        for row in reader:
            ObjectID = row[0]
            country = row[1]
            ISO3 = row[2]

            T_list = []
            total = 0
            count = 0
            for i in range(start_idx, end_idx + 1):
                if row[i] != "":
                    valor = float(row[i])
                    T_list.append(valor)
                    total += valor
                    count += 1
            T_media = total / count if count else 0

            dataset.append((ObjectID, country, ISO3, T_list, T_media))

    for ID, Pais, ISO3, T_list, T_media in dataset:
        avl.insertar_nodo(ID, Pais, ISO3, T_list, T_media)

    nodos_resultado = []

    while True:
        print("\n====== MENÚ AVL ======")
        print("1. Insertar un nodo (por ISO3 desde CSV)")
        print("2. Eliminar un nodo (por ISO3)")
        print("3. Buscar un nodo (por ISO3)")
        print("4. Búsquedas avanzadas")
        print("5. Mostrar recorrido por niveles")
        print("6. Operaciones sobre nodos encontrados")
        print("0. Salir")
        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            ISO3 = input("Ingrese el ISO3 del país a insertar: ")
            nodo, _, _ = avl.buscar_ISO(ISO3)
            if nodo:
                print(f"El acrónimo {ISO3} ya está en el árbol.")
            else:
                with open("/content/dataset_climate_change.csv", newline="") as f:
                    reader = csv.reader(f)
                    header = next(reader)
                    start_idx = header.index("F1961")
                    end_idx   = header.index("F2022")

                    encontrado = None
                    for row in reader:
                        if row[2] == ISO3:
                            ObjectID = row[0]
                            country = row[1]
                            T_list = []
                            total = 0
                            count = 0
                            for i in range(start_idx, end_idx + 1):
                                if row[i] != "":
                                    valor = float(row[i])
                                    T_list.append(valor)
                                    total += valor
                                    count += 1
                            T_media = total / count if count else 0
                            encontrado = (ObjectID, country, ISO3, T_list, T_media)
                            break

                if encontrado:
                    ID, Pais, ISO3, T_list, T_media = encontrado
                    avl.insertar_nodo(ID, Pais, ISO3, T_list, T_media)
                else:
                    print(f"El ISO3 {ISO3} no existe en el CSV.")

        elif opcion == "2":
            ISO3 = input("Ingrese ISO3 a eliminar: ")
            avl.eliminar_ISO(ISO3)

        elif opcion == "3":
            ISO3 = input("Ingrese ISO3 a buscar: ")
            nodo, padre, abuelo = avl.buscar_ISO(ISO3)
            if nodo:
                print("Nodo encontrado:")
                mostrar_nodo(nodo)
                nodos_resultado = [nodo]
            else:
                print("No encontrado.")

        elif opcion == "4":
            print("a) Año > promedio de todos los países en ese año")
            print("b) Año < promedio global (todos los años, todos los países)")
            print("c) T_media >= un valor dado")
            sub = input("Seleccione subopción: ")

            if sub == "a":
                year = int(input("Ingrese el año (ej. 1975): "))
                year_idx = year - 1961
                nodos_resultado = avl.buscar_mayores_promedio_anual(year_idx)

            elif sub == "b":
                year = int(input("Ingrese el año (ej. 1975): "))
                year_idx = year - 1961
                nodos_resultado = avl.buscar_menores_promedio_global(year_idx)

            elif sub == "c":
                limite = float(input("Ingrese límite: "))
                nodos_resultado = avl.buscar_media_mayor_que(limite)

            print(f"Se encontraron {len(nodos_resultado)} nodos:")
            for n in nodos_resultado:
                mostrar_nodo(n)

        elif opcion == "5":
            avl.recorrido_por_niveles()

        elif opcion == "6":
            if not nodos_resultado:
                print("Primero usa la opción 3 o 4 para obtener nodos.")
                continue

            print("Seleccione un nodo por índice:")
            for i, n in enumerate(nodos_resultado):
                print(f"{i}. {n.ISO3} ({n.Pais})")
            idx = int(input("Índice: "))
            if idx < 0 or idx >= len(nodos_resultado):
                print("Índice inválido.")
                continue
            nodo = nodos_resultado[idx]

            print("a) Nivel del nodo")
            print("b) Factor de balanceo")
            print("c) Padre")
            print("d) Abuelo")
            print("e) Tío")
            sub = input("Seleccione subopción: ")

            if sub == "a":
                def nivel(n, actual, lvl=1):
                    if not actual: return 0
                    if actual == n: return lvl
                    return nivel(n, actual.izquierda, lvl+1) or nivel(n, actual.derecha, lvl+1)
                print("Nivel:", nivel(nodo, avl.raiz))

            elif sub == "b":
                print("Balance:", avl.obtener_balance(nodo))

            elif sub == "c":
                _, padre, _ = avl.buscar(avl.raiz, nodo.ISO3, None, None)
                print("Padre:", padre.ISO3 if padre else None)

            elif sub == "d":
                _, padre, abuelo = avl.buscar(avl.raiz, nodo.ISO3, None, None)
                print("Abuelo:", abuelo.ISO3 if abuelo else None)

            elif sub == "e":
                _, padre, abuelo = avl.buscar(avl.raiz, nodo.ISO3, None, None)
                tio = avl.tio(padre, abuelo)
                print("Tío:", tio.ISO3 if tio else None)

        elif opcion == "0":
            print("Saliendo...")
            break
        else:
            print("Opción no válida.")

menu()


Nodo con acrónimo AFG insertado.
Nodo con acrónimo ALB insertado.
Nodo con acrónimo DZA insertado.
Nodo con acrónimo ASM insertado.
Nodo con acrónimo AND insertado.
Nodo con acrónimo AGO insertado.
Nodo con acrónimo AIA insertado.
Nodo con acrónimo ATG insertado.
Nodo con acrónimo ARG insertado.
Nodo con acrónimo ARM insertado.
Nodo con acrónimo ABW insertado.
Nodo con acrónimo AUS insertado.
Nodo con acrónimo AUT insertado.
Nodo con acrónimo AZE insertado.
Nodo con acrónimo BHS insertado.
Nodo con acrónimo BHR insertado.
Nodo con acrónimo BGD insertado.
Nodo con acrónimo BRB insertado.
Nodo con acrónimo BLR insertado.
Nodo con acrónimo BEL insertado.
Nodo con acrónimo BLZ insertado.
Nodo con acrónimo BEN insertado.
Nodo con acrónimo BTN insertado.
Nodo con acrónimo BOL insertado.
Nodo con acrónimo BIH insertado.
Nodo con acrónimo BWA insertado.
Nodo con acrónimo BRA insertado.
Nodo con acrónimo VGB insertado.
Nodo con acrónimo BRN insertado.
Nodo con acrónimo BGR insertado.
Nodo con a